In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.colors as pc
from modified_spectral_method import *
from modified_louvain_method import *
import cvxpy as cp
import copy


#### Coefficient Calculation Methods

In [11]:
# function to calculate proxy using the intersection method
def calculate_coefficients_intersection_method(prices_data, metadata, index_data, liquid_bucket, date, use_index = False):

    company_communities = metadata['Ticker'].to_list()
    metadata = metadata.set_index('Ticker')

    # prepare the data for the date
    prices_data = prices_data.loc[date,:]

    # prepare index data for the date
    index_data.rename(columns={'AsOf':'Date'}, inplace=True)
    index_data['Date'] = pd.to_datetime(index_data['Date'], format='%d/%b/%y')
    index_data = index_data.sort_values(by='Date', ascending=True)
    index_data = index_data[index_data['Date'] == date]
    index_spread = index_data['ConvSpread'].values[0]
    # print(f"Index spread for the date {date} is {index_spread}")

    # prepare the liquid bucket data for the date
    liquid_bucket_sector = liquid_bucket['Sector']
    liquid_bucket_country = liquid_bucket['Country']
    liquid_bucket_ratings = liquid_bucket['Rating']
    liquid_bucket_tickers = metadata[(metadata['Sector'] == liquid_bucket_sector) & (metadata['Country'] == liquid_bucket_country) & (metadata['AverageRating'] == liquid_bucket_ratings)].index.to_list()
    liquid_bucket_spread = prices_data[liquid_bucket_tickers].mean(axis=0)
    print(f"Liquid bucket spread for the date {date} is {liquid_bucket_spread}")

    if use_index:
        global_spread = index_spread
    else:
        global_spread = liquid_bucket_spread

    # create all possible combinations of the buckets
    unique_buckets = []

    for i in range(len(company_communities)):
        bucket = f'{metadata.loc[company_communities[i], 'Sector']}, {metadata.loc[company_communities[i], 'Country']}, {metadata.loc[company_communities[i], 'AverageRating']}'
        if bucket not in unique_buckets:
            unique_buckets.append(bucket)
    if not use_index:
        # remove the liquid bucket from the unique buckets
        unique_buckets.remove(f'{liquid_bucket_sector}, {liquid_bucket_country}, {liquid_bucket_ratings}')

    # prepare the prices data and the indicator matrix
    prices_data = prices_data.T.to_numpy()
    prices_data = prices_data.reshape(-1, 1)

    # create the indicator matrix
    indicators = np.zeros((len(company_communities), len(unique_buckets)))
    for i in range(len(company_communities)):
        # create string for the bucket
        bucket = f'{metadata.loc[company_communities[i], 'Sector']}, {metadata.loc[company_communities[i], 'Country']}, {metadata.loc[company_communities[i], 'AverageRating']}'
        if bucket in unique_buckets:
            j = unique_buckets.index(bucket)
            indicators[i, j] = 1

    # a_0
    a_0 = np.tile(global_spread, (len(company_communities), 1))

    # create optimization variables
    betas = cp.Variable(shape=(len(unique_buckets), 1))

    beta_contributions = indicators @ betas

    # Define the objective function
    objective = cp.Minimize(cp.norm(prices_data - a_0 - beta_contributions, "fro")**2)

    # Solve the optimization problem
    problem = cp.Problem(objective)
    problem.solve()

    # store the coefficients
    coefficients = pd.DataFrame({'bucket': unique_buckets, 'Coefficient': betas.value.flatten()})
    coefficients = coefficients.set_index('bucket')

    return coefficients

In [3]:
# function to calculate proxy using the CSRA community method
def calculate_proxy_coeff_csra_community(prices_data, communities, metadata, index_data, liquid_bucket, date, use_index = False):
    if not isinstance(communities[0], list):
        communities = [communities]
    coefficients = {}
    metadata = metadata.set_index('Ticker')
    # prepare the data for the date
    prices_data = prices_data.loc[date,:]

    # prepare index data for the date
    index_data.rename(columns={'AsOf':'Date'}, inplace=True)
    index_data['Date'] = pd.to_datetime(index_data['Date'], format='%d/%b/%y')
    index_data = index_data.sort_values(by='Date', ascending=True)
    index_data = index_data[index_data['Date'] == date]
    index_spread = index_data['ConvSpread'].values[0]
    # print(f"Index spread for the date {date} is {index_spread}")

    # prepare the liquid bucket data for the date
    liquid_bucket_sector = liquid_bucket['Sector']
    liquid_bucket_country = liquid_bucket['Country']
    liquid_bucket_ratings = liquid_bucket['Rating']
    liquid_bucket_tickers = metadata[(metadata['Sector'] == liquid_bucket_sector) & (metadata['Country'] == liquid_bucket_country) & (metadata['AverageRating'] == liquid_bucket_ratings)].index.to_list()
    liquid_bucket_spread = prices_data[liquid_bucket_tickers].mean(axis=0)

    # print(f"Liquid bucket spread for the date {date} is {liquid_bucket_spread}")

    if use_index:
        global_spread = index_spread
    else:
        global_spread = liquid_bucket_spread
    
    for community_number, community in enumerate(communities):

        # prepare the data for the community
        prices_data_community = prices_data[community]
        metadata_community = metadata.loc[community,:]

        sectors_community = metadata_community.loc[community, 'Sector'].unique().tolist()

        countries_community = metadata_community.loc[community, 'Country'].unique().tolist()

        ratings_community = metadata_community.loc[community, 'AverageRating'].unique().tolist()

        # remove the liquid bucket from the community
        if not use_index:
            if liquid_bucket_sector in sectors_community:
                sectors_community.remove(liquid_bucket_sector)
            if liquid_bucket_country in countries_community:
                countries_community.remove(liquid_bucket_country)
            if liquid_bucket_ratings in ratings_community:
                ratings_community.remove(liquid_bucket_ratings)

        prices_data_community = prices_data_community.T.to_numpy()
        prices_data_community = prices_data_community.reshape(-1, 1)

        # tranform the data to the log space
        prices_data_community_log = np.log(prices_data_community)
        # create the masks
        mask = np.zeros((prices_data_community.shape[0], len(sectors_community) + len(countries_community) + len(ratings_community)))

        for i in range(prices_data_community.shape[0]):
            if not use_index:
                if metadata_community.loc[community[i], 'Sector'] in sectors_community:
                    j = sectors_community.index(metadata_community.loc[community[i], 'Sector'])
                    mask[i, j] = 1
                if metadata_community.loc[community[i], 'Country'] in countries_community:
                    j = len(sectors_community) + countries_community.index(metadata_community.loc[community[i], 'Country']) - 1
                    mask[i, j] = 1
                if metadata_community.loc[community[i], 'AverageRating'] in ratings_community:
                    j = len(sectors_community) + len(countries_community) + ratings_community.index(metadata_community.loc[community[i], 'AverageRating']) - 1
                    mask[i, j] = 1


            else:    
                j = sectors_community.index(metadata_community.loc[community[i], 'Sector'])
                mask[i, j] = 1
                k = len(sectors_community) + countries_community.index(metadata_community.loc[community[i], 'Country'])  - 1
                mask[i, k] = 1
                l = len(sectors_community) + len(countries_community) + ratings_community.index(metadata_community.loc[community[i], 'AverageRating']) - 1

        # beta_0
        beta_0 = np.tile(np.log(global_spread), (prices_data_community.shape[1], 1))

        # Define optimization variables
        betas = cp.Variable(shape=(len(sectors_community) + len(countries_community) + len(ratings_community), 1))

        beta_contributions = mask @ betas

        # Define the objective function
        objective = cp.Minimize(cp.norm(prices_data_community_log - beta_0 -  beta_contributions, "fro")**2)


        # Solve the optimization problem
        problem = cp.Problem(objective)
        problem.solve()

        # store the coefficients
        sector_betas = betas.value[0:len(sectors_community)]
        country_betas = betas.value[len(sectors_community): len(sectors_community) + len(countries_community)]
        rating_betas = betas.value[len(sectors_community) + len(countries_community):]

        sectors_df = pd.DataFrame({'Name': sectors_community, 'Coefficient': sector_betas.flatten(), 'Type': 'Sector'},)
        countries_df = pd.DataFrame({'Name': countries_community, 'Coefficient': country_betas.flatten(), 'Type': 'Country'})
        ratings_df = pd.DataFrame({'Name': ratings_community, 'Coefficient': rating_betas.flatten(), 'Type': 'Rating'}) 

        # Combine DataFrames
        coefficients_df = pd.concat([sectors_df, countries_df, ratings_df], ignore_index=True)
        coefficients_df = coefficients_df.set_index('Name')
        coefficients[f'community_{community_number+1}'] = coefficients_df
    
    return coefficients

#### Proxy Calculation Methods

In [13]:
def calculate_proxy_intersection_method(ticker, metadata, coefficients, prices_data, index_data, liquid_bucket, date, use_index = False):
    metadata = metadata.set_index('Ticker')
    # prepare the data for the date
    prices_data = prices_data.loc[date,:]

    # prepare index data for the date
    index_data.rename(columns={'AsOf':'Date'}, inplace=True)
    index_data['Date'] = pd.to_datetime(index_data['Date'], format='%d/%b/%y')
    index_data = index_data.sort_values(by='Date', ascending=True)
    index_data = index_data[index_data['Date'] == date]
    index_spread = index_data['ConvSpread'].values[0]
    # print(f"Index spread for the date {date} is {index_spread}")

    # prepare the liquid bucket data for the date
    liquid_bucket_sector = liquid_bucket['Sector']
    liquid_bucket_country = liquid_bucket['Country']
    liquid_bucket_ratings = liquid_bucket['Rating']
    liquid_bucket_tickers = metadata[(metadata['Sector'] == liquid_bucket_sector) & (metadata['Country'] == liquid_bucket_country) & (metadata['AverageRating'] == liquid_bucket_ratings)].index.to_list()
    liquid_bucket_spread = prices_data[liquid_bucket_tickers].mean(axis=0)
    # print(f"Liquid bucket spread for the date {date} is {liquid_bucket_spread}")

    if use_index:
        global_spread = index_spread
    else:
        global_spread = liquid_bucket_spread
    company_bucket = f'{metadata.loc[ticker, 'Sector']}, {metadata.loc[ticker, 'Country']}, {metadata.loc[ticker, 'AverageRating']}'

    if company_bucket not in coefficients.index:
        print(f"Bucket not found for Company {ticker} ")
        return global_spread
    else:
        coefficient = coefficients.loc[company_bucket, 'Coefficient']

        proxy = coefficient + global_spread

    return proxy

In [5]:
def calculate_proxy_csra_community(ticker,  metadata, coefficients, ticker_community, prices_data, index_data, liquid_bucket, date, use_index = False):
    metadata = metadata.set_index('Ticker')
    # prepare the data for the date
    prices_data = prices_data.loc[date,:]

    # prepare index data for the date
    index_data.rename(columns={'AsOf':'Date'}, inplace=True)
    index_data['Date'] = pd.to_datetime(index_data['Date'], format='%d/%b/%y')
    index_data = index_data.sort_values(by='Date', ascending=True)
    index_data = index_data[index_data['Date'] == date]
    index_spread = index_data['ConvSpread'].values[0]
    # print(f"Index spread for the date {date} is {index_spread}")

    # prepare the liquid bucket data for the date
    liquid_bucket_sector = liquid_bucket['Sector']
    liquid_bucket_country = liquid_bucket['Country']
    liquid_bucket_ratings = liquid_bucket['Rating']
    liquid_bucket_tickers = metadata[(metadata['Sector'] == liquid_bucket_sector) & (metadata['Country'] == liquid_bucket_country) & (metadata['AverageRating'] == liquid_bucket_ratings)].index.to_list()
    liquid_bucket_spread = prices_data[liquid_bucket_tickers].mean(axis=0)
    # print(f"Liquid bucket spread for the date {date} is {liquid_bucket_spread}")

    if use_index:
        global_spread = np.log(index_spread)
    else:
        global_spread = np.log(liquid_bucket_spread)

    company_community = f'community_{ticker_community}'
    coefficients_ticker_community = coefficients[company_community]

    # get the coefficients for the sector, country and rating
    if metadata.loc[ticker, 'Sector'] in coefficients_ticker_community.index:
        sector_coefficient = coefficients_ticker_community.loc[metadata.loc[ticker, 'Sector'], 'Coefficient']
    else:
        sector_coefficient = 0
    
    if metadata.loc[ticker, 'Country'] in coefficients_ticker_community.index:
        country_coefficient = coefficients_ticker_community.loc[metadata.loc[ticker, 'Country'], 'Coefficient']
    else:
        country_coefficient = 0

    if metadata.loc[ticker, 'AverageRating'] in coefficients_ticker_community.index:
        rating_coefficient = coefficients_ticker_community.loc[metadata.loc[ticker, 'AverageRating'], 'Coefficient']
    else:
        rating_coefficient = 0

    proxy_log = sector_coefficient + country_coefficient + rating_coefficient + global_spread

    proxy = np.exp(proxy_log)

    return proxy

In [6]:
# load the prices data, index data and metadata 
prices_data = pd.read_csv('data/reshaped_data.csv')
prices_data['Date'] = pd.to_datetime(prices_data['Date'], format='%Y-%m-%d')
prices_data = prices_data.set_index('Date')

index_data = pd.read_csv('ITRAXX-Europe Timeseries 20241127.csv')

metadata = pd.read_csv('data/metadata.csv')
 
# perform community detection using the modified spectral method
correlation_matrix,T,N,company_names = create_correlation_matrix('data/eur_data_standardized_returns.csv')
C_g = calculate_C_g(correlation_matrix, T, N)
result_communities, company_communities, modularities = recursive_spectral_method(C_g, correlation_matrix, company_names, min_size=2, modularity_threshold=0.00001)
print(f"number of communities detected:{len(company_communities)}")


number of communities detected:3


In [12]:
# define the ticker to calculate the proxy for
ticker_proxy = 'AF'
# find the community to which the ticker belongs
ticker_community = None
for i, community in enumerate(company_communities):
    if ticker_proxy in community:
        ticker_community = i+1
        break

print(f"Calculating proxy for ticker {ticker_proxy}")
print(f'The ticker {ticker_proxy} belongs to the community {ticker_community}')

# define the liquid bucket
liquid_bucket = {'Sector': 'Financials', 'Country': 'United Kingdom', 'Rating': 6}
date = '2015-12-30'
date = pd.to_datetime(date, format='%Y-%m-%d') # convert the date to pandas datetime format

# get the actual spread for the ticker
actual_spread = prices_data.loc[date, ticker_proxy]
print(f'Actual spread for the ticker {ticker_proxy} is {actual_spread}')

# remove the ticker_proxy from the community data, prices data and metadata
company_communities_proxy_method = copy.deepcopy(company_communities)
print(company_communities_proxy_method[ticker_community-1])
company_communities_proxy_method[ticker_community-1].remove(ticker_proxy)
metadata_proxy_method = metadata[metadata['Ticker'] != ticker_proxy]
prices_data_proxy_method = prices_data.drop(columns= [ticker_proxy])

all_companies = [company for community in company_communities for company in community]
all_companies_proxy_method = [company for community in company_communities_proxy_method for company in community]

# calculate the coefficients and proxy using the intersection method
coefficients_intersection = calculate_coefficients_intersection_method(prices_data_proxy_method, metadata_proxy_method, index_data, liquid_bucket, date, use_index = False)
proxy_intersection = calculate_proxy_intersection_method(ticker_proxy, metadata, coefficients_intersection, prices_data, index_data, liquid_bucket, date, use_index = False)
# print(coefficients)
print(f'calculated proxy using the intersection method: {proxy_intersection}')

# calculate the coefficients and proxy using the CSRA method without the communities
coefficients_csra = calculate_proxy_coeff_csra_community(prices_data_proxy_method, all_companies_proxy_method, metadata_proxy_method, index_data, liquid_bucket, date, use_index = True)
# for key, value in coefficients.items():
#     print(key)
#     print(value)
#     print("")
proxy_csra = calculate_proxy_csra_community(ticker_proxy, metadata, coefficients_csra, 1, prices_data, index_data, liquid_bucket, date, use_index = True)
print(f'calculated proxy using the CSRA method: {proxy_csra}')

# calculate the coefficients and proxy using the CSRA community method
coefficients_csra_community = calculate_proxy_coeff_csra_community(prices_data_proxy_method, company_communities_proxy_method, metadata_proxy_method, index_data, liquid_bucket, date, use_index = True)
proxy_csra_community = calculate_proxy_csra_community(ticker_proxy, metadata, coefficients_csra_community, ticker_community, prices_data, index_data, liquid_bucket, date, use_index = True)
print(f'calculated proxy using the CSRA community method: {proxy_csra_community}')


Calculating proxy for ticker AF
The ticker AF belongs to the community 3
Actual spread for the ticker AF is 396.954622406639
['ABE', 'ABHLTD', 'ACAFP-CIB', 'ACEA', 'ADIG', 'AF', 'AF-AirFrance', 'AGASSA', 'AGS', 'AGS-Intl', 'AKZO-ICILD', 'ALCLCT', 'ALINDR', 'ALT', 'ALYD', 'AMROBK', 'ANGLIA', 'ARMLL-FRA', 'ARMLL-Fin', 'ASSAAB', 'ATCOA', 'AUST', 'AWLN-AngWatServ', 'AYLL', 'BAB', 'BANSAB', 'BCPN', 'BELG', 'BKIR', 'BKTSM', 'BNCBPI', 'BOCGP', 'BPBL', 'BRISA', 'BRITLD', 'BRYBDC', 'BRYBDC-Brew', 'BSFSHW', 'BWGPSK', 'BYLAN', 'CAJAME', 'CAWILD', 'CCBP-NATIXI', 'CCLL', 'CIRINT', 'CMFP', 'CTSGL', 'CXGD', 'CYPRUS', 'DAA', 'DBB', 'DENK', 'DEPFA-Bank', 'DEXGRP-CredLoc', 'DFHH', 'DMGOLN', 'DPW', 'DZBK', 'EDP-FinBV', 'ELEBEL', 'ELESM', 'ELIA', 'ELIAC', 'EMIG', 'ENDP', 'EON-PWGLN', 'EON-UK', 'ERGBA', 'ESS', 'EVRGSA', 'F-FCEBank', 'FERRUZ', 'FGPLN', 'FRTR', 'GFCFP', 'GROHE', 'HAMRSN', 'HAVAS', 'HBOS', 'HBOS-ScotBkPLC', 'HEI-HANSN', 'HEIANA-SCOT', 'HENAGK', 'HESLAN', 'HSBC', 'HSBC-HSBCBank', 'IBERDU-Intl'

In [8]:
def calculate_proxies_and_add_to_metadata(metadata, company_communities, prices_data, index_data, liquid_bucket, date):
    """
    Calculates proxy spreads and actual spreads for tickers in metadata and adds them as columns to a copy of metadata.

    Parameters:
    - metadata: pd.DataFrame, contains metadata including tickers.
    - company_communities: list of lists, each inner list is a community containing tickers.
    - prices_data: pd.DataFrame, price data with tickers as columns.
    - index_data: additional index-related data.
    - liquid_bucket: dict, liquid bucket information (e.g., Sector, Country, Rating).
    - date: str or pd.Timestamp, the date for which proxies are calculated.

    Returns:
    - metadata_with_proxies: pd.DataFrame, copy of metadata with calculated proxy and actual spread columns added.
    """
    # Ensure date is in pandas datetime format
    date = pd.to_datetime(date, format='%Y-%m-%d')
    
    # Create a copy of metadata to store results
    metadata_with_proxies = metadata.copy()
    metadata_with_proxies['ActualSpread'] = None
    metadata_with_proxies['ProxyIntersection'] = None
    metadata_with_proxies['ProxyCSRA'] = None
    metadata_with_proxies['ProxyCSRACommunity'] = None

    # Iterate over tickers in metadata
    for ticker_proxy in metadata['Ticker']:
        print(f"\nCalculating proxy for ticker: {ticker_proxy}")

        # Find the community to which the ticker belongs
        ticker_community = None
        for i, community in enumerate(company_communities):
            if ticker_proxy in community:
                ticker_community = i + 1
                break

        # Get the actual spread
        actual_spread = prices_data.loc[date, ticker_proxy]
        print(f"Actual spread for {ticker_proxy}: {actual_spread}")

        # Remove the ticker_proxy from the relevant data structures
        company_communities_proxy_method = copy.deepcopy(company_communities)
        company_communities_proxy_method[ticker_community - 1].remove(ticker_proxy)
        metadata_proxy_method = metadata[metadata['Ticker'] != ticker_proxy]
        prices_data_proxy_method = prices_data.drop(columns=[ticker_proxy])

        # Flatten company communities for proxy method
        all_companies_proxy_method = [company for community in company_communities_proxy_method for company in community]

        # Calculate proxies using intersection and CSRA methods
        try:
            coefficients_intersection = calculate_coefficients_intersection_method(
                prices_data_proxy_method, metadata_proxy_method, index_data, liquid_bucket, date, use_index=False
            )
            proxy_intersection = calculate_proxy_intersection_method(
                ticker_proxy, metadata, coefficients_intersection, prices_data, index_data, liquid_bucket, date, use_index=False
            )
        except Exception as e:
            print(f"Error calculating proxy using intersection method for {ticker_proxy}: {e}")
            proxy_intersection = None

        try:
            coefficients_csra = calculate_proxy_coeff_csra_community(
                prices_data_proxy_method, all_companies_proxy_method, metadata_proxy_method, index_data, liquid_bucket, date, use_index=True
            )
            proxy_csra = calculate_proxy_csra_community(
                ticker_proxy, metadata, coefficients_csra, 1, prices_data, index_data, liquid_bucket, date, use_index=True
            )
        except Exception as e:
            print(f"Error calculating proxy using CSRA method for {ticker_proxy}: {e}")
            proxy_csra = None

        try:
            coefficients_csra_community = calculate_proxy_coeff_csra_community(
                prices_data_proxy_method, company_communities_proxy_method, metadata_proxy_method, index_data, liquid_bucket, date, use_index=True
            )
            proxy_csra_community = calculate_proxy_csra_community(
                ticker_proxy, metadata, coefficients_csra_community, ticker_community, prices_data, index_data, liquid_bucket, date, use_index=True
            )
        except Exception as e:
            print(f"Error calculating proxy using CSRA community method for {ticker_proxy}: {e}")
            proxy_csra_community = None

        # Update the copied metadata with the results
        metadata_with_proxies.loc[metadata_with_proxies['Ticker'] == ticker_proxy, 'ActualSpread'] = actual_spread
        metadata_with_proxies.loc[metadata_with_proxies['Ticker'] == ticker_proxy, 'ProxyIntersection'] = proxy_intersection
        metadata_with_proxies.loc[metadata_with_proxies['Ticker'] == ticker_proxy, 'ProxyCSRA'] = proxy_csra
        metadata_with_proxies.loc[metadata_with_proxies['Ticker'] == ticker_proxy, 'ProxyCSRACommunity'] = proxy_csra_community

    return metadata_with_proxies


In [9]:
metadata_with_proxies = calculate_proxies_and_add_to_metadata(metadata, company_communities, prices_data, index_data, liquid_bucket, date)
metadata_with_proxies.to_csv('data/metadata_with_proxies.csv', index=False)


Calculating proxy for ticker: AAUK
Actual spread for AAUK: 984.6011
Bucket not found for Company AAUK 

Calculating proxy for ticker: ABE
Actual spread for ABE: 112.2037
Bucket not found for Company ABE 

Calculating proxy for ticker: ABHLTD
Actual spread for ABHLTD: 119.4848

Calculating proxy for ticker: ACAFP
Actual spread for ACAFP: 68.0217

Calculating proxy for ticker: ACAFP-CIB
Actual spread for ACAFP-CIB: 65.4582

Calculating proxy for ticker: ACCOR
Actual spread for ACCOR: 102.841

Calculating proxy for ticker: ACEA
Actual spread for ACEA: 87.3365

Calculating proxy for ticker: ADIG
Actual spread for ADIG: 72.96300000000001
Bucket not found for Company ADIG 

Calculating proxy for ticker: AEGON
Actual spread for AEGON: 77.548

Calculating proxy for ticker: AF
Actual spread for AF: 396.954622406639
Bucket not found for Company AF 

Calculating proxy for ticker: AF-AirFrance
Actual spread for AF-AirFrance: 85.2719
Bucket not found for Company AF-AirFrance 

Calculating proxy fo

c:\Users\dsvin\AppData\Local\Programs\Python\Python312\Lib\site-packages\cvxpy\problems\problem.py:1481: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Bucket not found for Company AUST 

Calculating proxy for ticker: AVLN
Actual spread for AVLN: 64.48660000000001

Calculating proxy for ticker: AWLN-AngWatServ
Actual spread for AWLN-AngWatServ: 98.0355

Calculating proxy for ticker: AXAF
Actual spread for AXAF: 55.9697

Calculating proxy for ticker: AYLL
Actual spread for AYLL: 158.3931

Calculating proxy for ticker: AZN
Actual spread for AZN: 38.3575
Bucket not found for Company AZN 

Calculating proxy for ticker: BAB
Actual spread for BAB: 151.147
Bucket not found for Company BAB 


c:\Users\dsvin\AppData\Local\Programs\Python\Python312\Lib\site-packages\cvxpy\problems\problem.py:1481: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(



Calculating proxy for ticker: BACR-Bank
Actual spread for BACR-Bank: 60.60570000000001

Calculating proxy for ticker: BACRED
Actual spread for BACRED: 129.9474

Calculating proxy for ticker: BAD
Actual spread for BAD: 67.63419999999999
Bucket not found for Company BAD 

Calculating proxy for ticker: BANSAB
Actual spread for BANSAB: 166.3289

Calculating proxy for ticker: BAPLC
Actual spread for BAPLC: 66.6246
Bucket not found for Company BAPLC 

Calculating proxy for ticker: BASFSE
Actual spread for BASFSE: 40.1395

Calculating proxy for ticker: BATSLN
Actual spread for BATSLN: 55.1769

Calculating proxy for ticker: BBVSM
Actual spread for BBVSM: 133.5554

Calculating proxy for ticker: BCPN
Actual spread for BCPN: 454.841
Bucket not found for Company BCPN 

Calculating proxy for ticker: BELG
Actual spread for BELG: 24.1429
Bucket not found for Company BELG 

Calculating proxy for ticker: BFCM
Actual spread for BFCM: 47.85239999999999

Calculating proxy for ticker: BKIR
Actual spread f

...


Calculating proxy for ticker: UU
Actual spread for UU: 83.05009999999999
Bucket not found for Company UU 

Calculating proxy for ticker: VEOLIA
Actual spread for VEOLIA: 68.1233
Bucket not found for Company VEOLIA 

Calculating proxy for ticker: VINCI
Actual spread for VINCI: 52.0098

Calculating proxy for ticker: VIRFIN
Actual spread for VIRFIN: 261.5415
Bucket not found for Company VIRFIN 

Calculating proxy for ticker: VIVNDI
Actual spread for VIVNDI: 75.7304
Bucket not found for Company VIVNDI 

Calculating proxy for ticker: VLOF
Actual spread for VLOF: 73.7707

Calculating proxy for ticker: VLVY
Actual spread for VLVY: 109.376

Calculating proxy for ticker: VLVY-Treasury
Actual spread for VLVY-Treasury: 115.7562

Calculating proxy for ticker: VOD
Actual spread for VOD: 92.9631

Calculating proxy for ticker: VW
Actual spread for VW: 151.6581


c:\Users\dsvin\AppData\Local\Programs\Python\Python312\Lib\site-packages\cvxpy\problems\problem.py:1481: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(



Calculating proxy for ticker: WEINBG
Actual spread for WEINBG: 294.2314455212508
Bucket not found for Company WEINBG 

Calculating proxy for ticker: WENL
Actual spread for WENL: 103.764

Calculating proxy for ticker: WINDAQ
Actual spread for WINDAQ: 396.6427000000002
Bucket not found for Company WINDAQ 

Calculating proxy for ticker: WOLKLU
Actual spread for WOLKLU: 36.3458
Bucket not found for Company WOLKLU 

Calculating proxy for ticker: WPPGRP-2005
Actual spread for WPPGRP-2005: 50.0136


c:\Users\dsvin\AppData\Local\Programs\Python\Python312\Lib\site-packages\cvxpy\problems\problem.py:1481: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(



Calculating proxy for ticker: YORPOW
Actual spread for YORPOW: 66.95490000000001

Calculating proxy for ticker: ZINCO
Actual spread for ZINCO: 41.1356
Bucket not found for Company ZINCO 
